# Summer Reccomender Algo Research 
> Google colab is so slow bruh
- toc: true
- title: Pain and more pain
- author: Alex Lu
- badges: true
- comments: true
- categories: [jupyter,interests]

## Import all of our required modules

In [ ]:
## Init.py

import pandas as pd
import numpy as np
!pip install scikit-surprise
from surprise import Dataset
from surprise import Reader
import scipy

## Import and process our data

In [ ]:
## Data.py

## Importing our ratings data
ratings = pd.read_csv("rating.csv")
anime = pd.read_csv("anime.csv")

## removing entries with no ratings and resetting indices
ratings = ratings.head(125000)
## Apply to_numeric and removing all NaN values 
ratings = ratings.apply(pd.to_numeric, errors='coerce')
ratings = ratings.dropna()

## Declaring some stuff for KNN calculation
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(ratings[["user_id", "anime_id", "rating"]], reader)
ratings

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1
...,...,...,...
124995,1272,4382,10
124996,1272,4672,5
124997,1272,4792,6
124998,1272,4898,8


**KNN**

In [ ]:
ratings[["user_id", "anime_id", "rating"]]

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1
...,...,...,...
124995,1272,4382,10
124996,1272,4672,5
124997,1272,4792,6
124998,1272,4898,8


In [ ]:
#recommender.py

from surprise import KNNWithMeans

sim_options = {
    "name": "msd",
    "min_support": 3,
    "user_based": False,  # Compute  similarities between items
}
algo = KNNWithMeans(sim_options=sim_options)

user_id  = input("User Id   >>>  ")
anime_id = input("Anime Id  >>>  ")

trainingSet = data.build_full_trainset()
algo.fit(trainingSet)
prediction = algo.predict(int(user_id), int(anime_id))



from google.colab import output
output.clear()
print("User {0} would rate anime number {1}".format(str(user_id), str(anime_id)))
prediction.est

User 3 would rate anime number 101


6.695240884293928

In [ ]:
#recommender.py

from surprise import KNNWithZScore

sim_options = {
    "name": "msd",
    "min_support": 3,
    "user_based": False,  # Compute  similarities between items
}
algo = KNNWithZScore(sim_options=sim_options)

user_id  = input("User Id   >>>  ")
anime_id = input("Anime Id  >>>  ")

trainingSet = data.build_full_trainset()
algo.fit(trainingSet)
prediction = algo.predict(int(user_id), int(anime_id))
from google.colab import output
output.clear()
print("User {0} would rate anime number {1}".format(str(user_id), str(anime_id)))
prediction.est

User 3 would rate anime number 101


6.702780614370237

**SVD algorithm**

Basic algorithms
NormalPredictor 
NormalPredictor algorithm predicts a random rating based on the distribution of the training set, which is assumed to be normal. This is one of the most basic algorithms that do not do much work.
BaselineOnly
BaselineOnly algorithm predicts the baseline estimate for given user and item.
k-NN algorithms
KNNBasic
KNNBasic is a basic collaborative filtering algorithm.
KNNWithMeans
KNNWithMeans is basic collaborative filtering algorithm, taking into account the mean ratings of each user.
KNNWithZScore
KNNWithZScore is a basic collaborative filtering algorithm, taking into account the z-score normalization of each user.
KNNBaseline
KNNBaseline is a basic collaborative filtering algorithm taking into account a baseline rating.
Matrix Factorization-based algorithms
SVD
SVD algorithm is equivalent to Probabilistic Matrix Factorization
SVDpp
The SVDpp algorithm is an extension of SVD that takes into account implicit ratings.
NMF
NMF is a collaborative filtering algorithm based on Non-negative Matrix Factorization. It is very similar with SVD.
Slope One
SlopeOne is a straightforward implementation of the SlopeOne algorithm.
Co-clustering
Coclustering is a collaborative filtering algorithm based on co-clustering.
We use “rmse” as our accuracy metric for the predictions.

**KNNWithMeans**

In [ ]:
#acc1.test

from surprise import KNNWithMeans
from surprise import Dataset
from surprise.model_selection import GridSearchCV

data = Dataset.load_from_df(ratings[["user_id", "anime_id", "rating"]], reader)
sim_options = {
    "name": ["msd", "cosine"],
    "min_support": [3, 4, 5],
    "user_based": [False, True],
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNWithMeans, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])
print(gs.best_score["mae"])
print(gs.best_params["mae"])

# Cleaned
# rmse 1.1961042065214869
# {'sim_options': {'name': 'msd', 'min_support': 3, 'user_based': True}}
# mae 0.9066350800152176
#{'sim_options': {'name': 'msd', 'min_support': 3, 'user_based': False}}

# Original
#2.300630451838356
#{'sim_options': {'name': 'msd', 'min_support': 4, 'user_based': True}}
#1.6207772901276576
#{'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': True}}

# runtime 4m 28s

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

**KNNWithZScore**

In [ ]:
#acc2.test

from surprise import KNNWithZScore
from surprise import Dataset
from surprise.model_selection import GridSearchCV

data = Dataset.load_from_df(ratings[["user_id", "anime_id", "rating"]], reader)
sim_options = {
    "name": ["msd", "cosine"],
    "min_support": [3, 4, 5],
    "user_based": [False, True],
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNWithZScore, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])
print(gs.best_score["mae"])
print(gs.best_params["mae"])

# Cleaned
#rmse 1.2200524149502934
#{'sim_options': {'name': 'msd', 'min_support': 3, 'user_based': True}}
#mae 0.9071607750307225
#{'sim_options': {'name': 'msd', 'min_support': 4, 'user_based': True}}

# Original
#2.3157416009374785
#{'sim_options': {'name': 'cosine', 'min_support': 4, 'user_based': True}}
#1.608562753271584
#{'sim_options': {'name': 'cosine', 'min_support': 4, 'user_based': True}}
#runtime 9m 28s

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

**BaselineOnly**

In [ ]:
#acc3.test

from surprise import BaselineOnly
from surprise import Dataset
from surprise.model_selection import GridSearchCV
# Misc
data = Dataset.load_from_df(ratings[["user_id", "anime_id", "rating"]], reader)
bsl_options = {'method': ['als', 'sgd'],
               'n_epochs': [1,5,10],
               'reg_u': [12],
               'reg_i': [5]
               }

param_grid = {"bsl_options": bsl_options}

gs = GridSearchCV(BaselineOnly, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])
print(gs.best_score["mae"])
print(gs.best_params["mae"])
# Original
# rmse 1.1937039196145423
#{'bsl_options': {'method': 'als', 'n_epochs': 10, 'reg_u': 12, 'reg_i': 5}}
# mae 0.9073417853583354
#{'bsl_options': {'method': 'als', 'n_epochs': 10, 'reg_u': 12, 'reg_i': 5}}

# Cleaned
#2.2457828423033965
#{'bsl_options': {'method': 'sgd', 'n_epochs': 10, 'reg_u': 12, 'reg_i': 5}}
#1.6065790249351644
#{'bsl_options': {'method': 'sgd', 'n_epochs': 10, 'reg_u': 12, 'reg_i': 5}}

# runtime 12s

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
Estimating biases using sgd...
2.2457828423033965
{'bsl_options': {'method': 'sgd', 'n_epochs': 10, 'reg_u': 12, 'reg_i': 5}}
1.6065790249351644
{'bsl_options': {'method': 'sgd', 'n_epochs': 10, 'reg_u': 12, 'reg_i': 5}}


**SVD**

In [ ]:
#acc4.test
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import GridSearchCV
# Matrix facotization
sim_options = {
    "n_epochs": [5,10],
    "lr_all": [0.002, 0.005],
    "reg_all": [0.4,0.6]
}

gs = GridSearchCV(SVD, sim_options, measures=["rmse", "mae"], cv=3)

gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])
print(gs.best_score["mae"])
print(gs.best_params["mae"])
# Original
#rmse 1.2370006802584665
#{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}
#mae 0.9457554878434046
#{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}

# Cleaned
#2.3105571863770495
#{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}
#1.7100669607848105
#{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}
#runtime 1m 44s

2.3105571863770495
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}
1.7100669607848105
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


**KNNBasic**

In [ ]:
#acc4.test
from surprise import KNNBasic
from surprise import Dataset
from surprise.model_selection import GridSearchCV
# Clustering
sim_options = {
    "name": ["msd", "cosine"],
    "k": [20,40],
    "min_k": [5, 7],
    "verbose": [True]
}


gs = GridSearchCV(KNNBasic, sim_options, measures=["rmse", "mae"], cv=3)

gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])
print(gs.best_score["mae"])
print(gs.best_params["mae"])
# Cleaned 
#rmse 1.3169600252251181
#{'name': 'msd', 'k': 20, 'min_k': 5, 'verbose': True}
#mae 0.9971319510312137
#{'name': 'msd', 'k': 20, 'min_k': 5, 'verbose': True}

# Original
#2.61306062703497
#{'name': 'msd', 'k': 20, 'min_k': 5, 'verbose': True}
#1.7634939088675168
#{'name': 'msd', 'k': 20, 'min_k': 5, 'verbose': True}
# 4m 53 s

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

**NMF**

In [ ]:
#acc5.test
from surprise import NMF
from surprise import Dataset
from surprise.model_selection import GridSearchCV
# Matrix Factorization
sim_options = {
    "n_epochs": [5,10,15,20],
    "biased": [True,False],
    "reg_pu": [0.01, 0.06, 0.5, 1],
    "reg_qi": [0.01, 0.06, 0.5, 1],
    "verbose": [True]
}

gs = GridSearchCV(NMF, sim_options, measures=["rmse", "mae"], cv=3)

gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])
print(gs.best_score["mae"])
print(gs.best_params["mae"])

# Cleaned
#1.1889671285702674
#{'n_epochs': 20, 'biased': True, 'reg_pu': 0.01, 'reg_qi': 1, 'verbose': True}
#0.9011699311141764
#{'n_epochs': 20, 'biased': True, 'reg_pu': 0.01, 'reg_qi': 1, 'verbose': True}

# Original
#2.2452077636572434
#{'n_epochs': 10, 'biased': True, 'reg_pu': 0.5, 'reg_qi': 0.5, 'verbose': True}
#1.6074999998727018
#{'n_epochs': 20, 'biased': True, 'reg_pu': 1, 'reg_qi': 1, 'verbose': True}
#runtime 11m 25s

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 0
Processing epoch 1
Processing e

**Normal Predictor**

In [ ]:
#acc6.test
from surprise import NormalPredictor
from surprise import Dataset
from surprise.model_selection import GridSearchCV
# Misc
sim_options = {
}

gs = GridSearchCV(NormalPredictor, sim_options, measures=["rmse", "mae"], cv=3)

gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])
print(gs.best_score["mae"])
print(gs.best_params["mae"])
# Cleaned
#rmse 2.0923401858242667
# {}
#mae 1.6582413347539269
# {}

# Original
#4.876980925979471
#{}
#3.9231832724710536
#{}
#runtime 2s

4.876980925979471
{}
3.9231832724710536
{}


**SlopeOne**

In [ ]:
#acc7.test
from surprise import SlopeOne
from surprise import Dataset
from surprise.model_selection import GridSearchCV
# Clustering
param_grid = {
}
gs = GridSearchCV(SlopeOne, param_grid, measures=["rmse", "mae"], cv=3)

gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])
print(gs.best_score["mae"])
print(gs.best_params["mae"])

# Cleaned
#1.2261258397066668
#{}
#0.9248136286120494
#{}

# Original
#2.3148602570198133
#{}
#1.646961825093937
#{}
#runtime 22s

2.3148602570198133
{}
1.646961825093937
{}


**CoClustering**

In [ ]:
#acc8.test
from surprise import CoClustering
from surprise import Dataset
from surprise.model_selection import GridSearchCV
# Clustering
sim_options = {
    "n_cltr_u": [2,3,4],
    "n_cltr_i": [2,3,4],
    "n_epochs": [10,15,20],
    "verbose" : [True]
}

gs = GridSearchCV(CoClustering, sim_options, measures=["rmse", "mae"], cv=3)

gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])
print(gs.best_score["mae"])
print(gs.best_params["mae"])

# Cleaned
#1.2533606254732572
#{'n_cltr_u': 4, 'n_cltr_i': 2, 'n_epochs': 10, 'verbose': True}
#0.9453075653296411
#{'n_cltr_u': 4, 'n_cltr_i': 2, 'n_epochs': 10, 'verbose': True}

# Original
#2.3921675151385458
#{'n_cltr_u': 3, 'n_cltr_i': 2, 'n_epochs': 20, 'verbose': True}
#1.687391909272779
#{'n_cltr_u': 4, 'n_cltr_i': 2, 'n_epochs': 15, 'verbose': True}
#runtime 10m 37s

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Process

**KNNBaseline**


In [ ]:
#acc9.test
# Clustering
from surprise import KNNBaseline
from surprise import Dataset
from surprise.model_selection import GridSearchCV

data = Dataset.load_from_df(ratings[["user_id", "anime_id", "rating"]], reader)
sim_options = {
    "name": ["msd", "cosine"],
    "min_support": [3, 4, 5],
    "user_based": [False, True],
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNBaseline, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])
print(gs.best_score["mae"])
print(gs.best_params["mae"])
# Cleaned
#1.1832580437001543
#{'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': False}}
#0.8893399468276982
#{'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': False}}

# Original
#2.2179062356553265
#{'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': False}}
#1.54544242417866
#{'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': False}}
#runtime 5m 15s

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matr

**SVDpp**

In [ ]:
#acc10.test
# Matrix facotrization
from surprise import SVDpp
from surprise import Dataset
from surprise.model_selection import GridSearchCV

data = Dataset.load_from_df(ratings[["user_id", "anime_id", "rating"]], reader)
param_grid = {
    "n_epochs": [5,10],
    "lr_all": [0.002, 0.005],
    "reg_all": [0.4,0.6]
}


gs = GridSearchCV(SVDpp, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

print(gs.best_score["rmse"])
print(gs.best_params["rmse"])
print(gs.best_score["mae"])
print(gs.best_params["mae"])
# Cleaned
#1.236439713814388
#{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}
#0.9482547980069017
#{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}

# Original
#2.30757862188976
#{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}
#1.6811210573467745
#{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}
#runtime 27m 43s

2.30757862188976
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}
1.6811210573467745
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}
